In [1]:
import numpy as np
import pandas as pd

from sklearn import model_selection
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
#from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.ensemble import GradientBoostingClassifier
#import xgboost as xgb
#from xgboost.sklearn import XGBClassifier

#time
from datetime import datetime
from datetime import timedelta

import jieba
import jieba.analyse
jieba.set_dictionary('dict.idkrsi.txt')     # 改預設字典
jieba.analyse.set_stop_words("stopword.goatwang.kang.txt") #指定stopwords字典

# get data
# ! conda install pandas-datareader s
#import pandas_datareader as pdr

# visual
# ! pip install mpl-finance
#import matplotlib.pyplot as plt
#import mpl_finance as mpf

#import seaborn as sns


# https://github.com/mrjbq7/ta-lib
# ! pip install ta-lib
#import talib

In [2]:
df_bbs = pd.read_csv("bda2019_dataset/bbs2.csv",encoding="utf-8")
df_forum = pd.read_csv("bda2019_dataset/forum2.csv",encoding="utf-8")
df_news = pd.read_csv("bda2019_dataset/news2.csv",encoding="utf-8")
df_news['comment_count']=0

df_article = pd.concat([df_forum, df_bbs, df_news])  #三個合併
del df_bbs, df_forum, df_news

df_article['post_time'] = pd.to_datetime(df_article['post_time'])
df_article['post_time2'] = df_article['post_time'].dt.date   # .dt.date用在dataframe  .date()用在一個 #只留日期
#df_article['label'] = 'even'
df_article['content'] = df_article['content'].astype(str).str.replace(',' , ' ').str.replace('\n' , ' ').str.replace('"' , ' ').str.replace("'" , ' ')
df_article['title'] = df_article['title'].astype(str).str.replace(',' , ' ').str.replace('\n' , ' ').str.replace('"' , ' ').str.replace("'" , ' ')
df_article = df_article.sort_values(by=['post_time']).reset_index(drop=True) # 用post_time排序 # 在重設index
df_article.head(2)
#df_article2 = df_article[['post_time2','title','content']]

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/ipykernel_launcher.py:6: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2
0,鉅亨網,0,鉅亨網編譯郭照青這一年，獲利真是難上難。<BR>根據CNBC分析報導，興業銀行的數據顯示，2...,1451580107540_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-01 00:06:00,重大要聞,yahoo股市,淪落的2015 竟是78年來最難獲利的一年,2016-01-01
1,Bokolo,10,1.原文連結(必須檢附)： http://magazine.chinatimes.com/m...,1451580710262_PTT02R,bbs,http://www.ptt.cc/bbs/Stock/M.1451580509.A.722...,2016-01-01 00:48:25,Stock,Ptt,[新聞] 第三方支付 台灣普及關鍵年,2016-01-01


In [3]:
df_TWSE2018 = pd.read_csv("bda2019_dataset/TWSE2018.csv",encoding="utf-8")
df_TWSE2017 = pd.read_csv("bda2019_dataset/TWSE2017.csv",encoding="utf-8")
df_TWSE2016 = pd.read_csv("bda2019_dataset/TWSE2016.csv",encoding="utf-8")

df_TWSE = pd.concat([df_TWSE2016, df_TWSE2017, df_TWSE2018])  #三年合併
del df_TWSE2016, df_TWSE2017, df_TWSE2018

# ['開盤價(元)', '最高價(元)', '最低價(元)', '收盤價(元)', '成交量(千股)', '成交值(千元)', '成交筆數(筆)', '流通在外股數(千股)', '本益比-TSE', '股價淨值比-TSE']
df_TWSE['證券代碼'] = df_TWSE['證券代碼'].astype(str)
df_TWSE['年月日'] = pd.to_datetime(df_TWSE['年月日'])
df_TWSE['開盤價(元)'] = df_TWSE['開盤價(元)'].str.replace(',' , '').astype('float64') # 1,000 to 1000 to float
df_TWSE['最高價(元)'] = df_TWSE['最高價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['最低價(元)'] = df_TWSE['最低價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['收盤價(元)'] = df_TWSE['收盤價(元)'].str.replace(',' , '').astype('float64')
df_TWSE['成交量(千股)'] = df_TWSE['成交量(千股)'].str.replace(',' , '').astype('float64')
df_TWSE['成交值(千元)'] = df_TWSE['成交值(千元)'].str.replace(',' , '').astype('float64')
df_TWSE['成交筆數(筆)'] = df_TWSE['成交筆數(筆)'].str.replace(',' , '').astype('int64')
df_TWSE['流通在外股數(千股)'] = df_TWSE['流通在外股數(千股)'].str.replace(',' , '').astype('float64')
df_TWSE['本益比-TSE'] = df_TWSE['本益比-TSE'].str.replace(',' , '').astype('float64')
df_TWSE['股價淨值比-TSE'] = df_TWSE['股價淨值比-TSE'].astype('float64')
df_TWSE.head(2) 

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,1101 台泥,2016-12-30,29.09,29.64,28.93,29.64,6121.0,213121.0,2939,3692175.0,20.56,1.23
1,1102 亞泥,2016-12-30,24.91,24.91,24.67,24.81,3347.0,88078.0,1227,3361447.0,31.75,0.73


In [4]:
# 選那家股票
#company_name = '國巨'
company_name = '奇力新'


# 文章包含那家字
#company_words = '被動元件|積層陶瓷電容|MLCC|電感|晶片電阻|車用電子|凱美|同欣電|大毅|君耀|普斯|國巨'
company_words = '被動元件|積層陶瓷電容|MLCC|電感|晶片電阻|車用電子|飛磁|旺詮|美磊|美桀|向華科技|奇力新'

# 漲跌幾%
PA = 0.05

# even幾%
PAE = 0.003

In [5]:
# 用日期排序 再把index重排
#2327 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('國巨')].sort_values(by=['年月日']).reset_index(drop=True)
#2456 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('奇力新')].sort_values(by=['年月日']).reset_index(drop=True)
#2478 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('大毅')].sort_values(by=['年月日']).reset_index(drop=True)
#6271 
#df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains('同欣電')].sort_values(by=['年月日']).reset_index(drop=True)

df_trend = df_TWSE[df_TWSE['證券代碼'].str.contains(company_name)].sort_values(by=['年月日']).reset_index(drop=True)
del df_TWSE
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46


In [6]:
##增欄位:fluctuation幅度 tag漲跌平
df_trend['fluctuation'] = 0.0
df_trend['tag']='--'
df_trend['closeshift'] = 0.0
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32,0.0,--,0.0
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46,0.0,--,0.0


In [7]:
# ##增欄位:fluctuation幅度 tag漲跌平
# df_trend['fluctuation'] = 0.0
# df_trend['tag']='--'

# ###計算漲跌
# for index, row in df_trend.iterrows():
#     try:
#         margin =(float(df_trend.loc[index,'收盤價(元)']) - float(df_trend.loc[index-1,'收盤價(元)']) )/ float(df_trend.loc[index-1,'收盤價(元)'])
#         df_trend.loc[index,'fluctuation']=margin
#         if margin >=0.03:
#             df_trend.loc[index,'tag']='up'
#         elif margin <= -0.03:
#             df_trend.loc[index,'tag']='down'
#         else:
#             df_trend.loc[index,'tag']='even'
#     except:
#         continue

In [8]:
df_trend['closeshift'] = df_trend['收盤價(元)'].shift(periods=1)#.fillna(value=0.0, inplace=True)
#df_trend['closeshift'].fillna(value= 0.0, inplace=True)
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32,0.0,--,NaN
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46,0.0,--,80.2


In [9]:
df_trend['fluctuation'] = (df_trend['收盤價(元)'] - df_trend['closeshift']) / df_trend['closeshift']
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32,NaN,--,NaN
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46,0.060599,--,80.2


In [10]:
df_trend.loc[df_trend['fluctuation'] >=  PA, 'tag'] = 'up'
df_trend.loc[df_trend['fluctuation'] <= -PA, 'tag'] = 'down'
df_trend.loc[(df_trend['fluctuation'] >= -PAE) & (df_trend['fluctuation'] <= PAE), 'tag'] = 'even'
df_trend.head(2)

,證券代碼,年月日,開盤價(元),最高價(元),最低價(元),收盤價(元),成交量(千股),成交值(千元),成交筆數(筆),流通在外股數(千股),本益比-TSE,股價淨值比-TSE,fluctuation,tag,closeshift
0,2456 奇力新,2016-01-04,80.44,81.74,78.18,80.20,4389.0,297206.0,2710,126969.0,15.89,2.32,NaN,--,NaN
1,2456 奇力新,2016-01-05,80.55,85.06,79.25,85.06,6701.0,468760.0,4200,126969.0,16.85,2.46,0.060599,up,80.2


In [11]:
len(df_trend[df_trend['tag']=='up'])

37

In [12]:
len(df_trend[df_trend['tag']=='down'])

39

In [13]:
len(df_trend[df_trend['tag']=='even'])

90

In [14]:
#df_company = df_article[ df_article['content'].str.contains('國巨')]   # df 某欄位 string contains "國巨"
#df_company = df_article[ df_article['content'].str.contains('奇力新')] 
#df_company = df_article[ df_article['content'].str.contains('大毅')]  
#df_company = df_article[ df_article['content'].str.contains('同欣電 ')]
df_company = df_article[ df_article['content'].str.contains(company_words)]   
print(len(df_company))
del df_article
df_company.head(2)

10553


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2
30,時報資訊,0,台股2015年下半年跌勢洶洶，全年跌幅達10.14％，但12檔具利多題材個股，股價多頭氣盛，...,1451618809219_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-01 10:59:00,重大要聞,yahoo股市,《熱門族群》12檔話題當紅股，完美封關,2016-01-01
97,鉅亨網,0,鉅亨網記者王莞甯 台北台股2015年封關日最後一檔有神秘買盤進場，強拉加權指數上漲39點，元...,1451715108530_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-02 13:50:00,重大要聞,yahoo股市,錢冠州：總統大選不再是大變數 元月行情伺機而動！,2016-01-02


In [15]:
stopwords=list()
with open('stopword.goatwang.kang.txt', 'r',encoding='utf-8') as data:
    for stopword in data:
        stopwords.append(stopword.strip('\n'))

In [16]:
# 'content'全部切詞
corpus = []  # array
for index, row in df_company.iterrows():  
    not_cut = df_company.loc[index,'content']
#     not_cut = row['description']                    # 跟上一行一樣意思
    seg_generator = jieba.cut(not_cut, cut_all=False)  # genarator
    seglist = list(seg_generator)                     # 整篇文章string切出來的list
#    seglist = list(filter(lambda a: a not in stopwords and a != '\n', seglist ))  #去除停用詞 #未必需要這步驟
    corpus.append(' '.join(seglist))          # ' '.join(seg_generator)也可

df_company["content2"]=corpus  

Building prefix dict from /home/barry/projects/stock/dict.idkrsi.txt ...
Loading model from cache /tmp/jieba.ufc1c7fa02eb193cee2bb78b34d63c017.cache
Loading model cost 0.331 seconds.
Prefix dict has been built succesfully.


In [17]:
df_company.head(2)

,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2
30,時報資訊,0,台股2015年下半年跌勢洶洶，全年跌幅達10.14％，但12檔具利多題材個股，股價多頭氣盛，...,1451618809219_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-01 10:59:00,重大要聞,yahoo股市,《熱門族群》12檔話題當紅股，完美封關,2016-01-01,台股 2015 年 下半年 跌勢 洶洶 ， 全年 跌幅 達 10.14 ％ ， 但 12 檔...
97,鉅亨網,0,鉅亨網記者王莞甯 台北台股2015年封關日最後一檔有神秘買盤進場，強拉加權指數上漲39點，元...,1451715108530_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-02 13:50:00,重大要聞,yahoo股市,錢冠州：總統大選不再是大變數 元月行情伺機而動！,2016-01-02,鉅亨網 記者 王莞 甯 台北 台股 2015 年 封關日 最後 一檔 有 神秘 買盤 進...


In [18]:
df_trend.loc[2,'年月日'].date() +  timedelta(days=-1) == df_trend.loc[1,'年月日'].date()

True

In [19]:
df_trend.loc[5,'年月日'].date() +  timedelta(days=-1) == df_trend.loc[4,'年月日'].date()

False

In [20]:
d = df_trend.loc[1,'年月日'].date() - df_trend.loc[ 1-1 ,'年月日'].date() #相減差幾天
d

datetime.timedelta(1)

In [21]:
d.days #只取天數

1

In [22]:
int(d.days) #幾天 轉整數

1

In [23]:
df_trend.loc[3,'年月日'].date() 

datetime.date(2016, 1, 7)

In [24]:
df_company[ df_company['post_time2'] == df_trend.loc[3,'年月日'].date() ].head() # 某欄位 == n 的 全部撈出來

,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2
999,Moneydj理財網,0,1.週三台股在美股重挫以及陸股再度啟動融斷機制之下，指數重挫點，指數跌破十年線，所幸尾盤大型...,1452156767683_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-07 16:29:00,重大要聞,yahoo股市,日盛投顧：台股短線有反彈機會,2016-01-07,1 . 週三 台股 在 美股 重挫 以及 陸股 再度 啟動 融斷 機制 之下 ， 指數 重挫...


In [25]:
# # 演算法 
# for index, row in df_2327.iterrows():
#     try:
#         if df_2327.loc[index,'年月日'].date() +  timedelta(days=-1) == df_2327.loc[index-1,'年月日'].date():                        
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             # 如果股票前一筆差1天  # 那前1天的文章標上當天的漲跌   
        
#         elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-2) == df_2327.loc[index-1,'年月日'].date():           
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-2), 'label'] = df_2327.loc[index,'tag']
#             # 如果股票前一筆差2天 #那前2天的文章標上當天的漲跌
            
#         elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-3) == df_2327.loc[index-1,'年月日'].date():
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-2), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-3), 'label'] = df_2327.loc[index,'tag']
#         elif df_2327.loc[index,'年月日'].date() +  timedelta(days=-4) == df_2327.loc[index-1,'年月日'].date():
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-1), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-2), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-3), 'label'] = df_2327.loc[index,'tag']
#             df_forum.loc[df_forum['post_time2'] ==  df_2327.loc[index,'年月日'].date() +  timedelta(days=-4), 'label'] = df_2327.loc[index,'tag']


#     except:
#         continue

In [26]:
# 看所有相差的天數
# for index, row in df_2327.iterrows():
#     try:
#         n = df_2327.loc[index,'年月日'].date() - df_2327.loc[index-1,'年月日'].date()
#         print(n)
#     except:
#         continue

# 最多12天

In [27]:
# 如果股票前一筆差n天  # 那前n天的文章標上當天的漲跌 
df_company['label5566']='--'
for index, row in df_trend.iterrows():
    try:
        n = int((df_trend.loc[index,'年月日'].date() - df_trend.loc[index-1,'年月日'].date()).days ) # 差幾個datetime # 轉天數 # 再轉整數
#        print(n)

        for i in range(1, n+1):
#            print(i)
            df_company.loc[df_company['post_time2'] ==  df_trend.loc[index,'年月日'].date() +  timedelta(days=-i), 'label5566'] = df_trend.loc[index,'tag']
    except:
        continue

In [28]:
print(len(df_company[df_company['label5566']=='down']))
df_company[df_company['label5566']=='down'].head(2)

1284


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2,label5566
1522,財訊快報,0,◆全台勞檢啟動，11類行業小心，其中284家電子零組件企業，實施全面普查。<BR> ◆海外分...,1452472319234_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-11 08:05:00,重大要聞,yahoo股市,工商時報晨訊,2016-01-11,◆ 全台 勞檢 啟動 ， 11 類 行業 小心 ， 其中 284 家 電子 零組件 企業 ，...,down
1530,時報資訊,0,陸股暫停熔斷機制，減緩市場對於系統性風險的疑慮，法人指出，台股短線壓力仍大，若結構出現惡化現...,1452472329341_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-11 08:15:00,重大要聞,yahoo股市,《基金》全球經濟趨緩，慎選台股標的,2016-01-11,陸股 暫停 熔斷 機制 ， 減緩 市場 對於 系統性 風險 的 疑慮 ， 法人 指出 ， 台...,down


In [29]:
print(len(df_company[df_company['label5566']=='up']))
df_company[df_company['label5566']=='up'].head(2)

891


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2,label5566
222,財訊快報,0,◆千億科技預算，張善政喊加碼，主計總處初步匡列1 061億、年增3%，可能再調高。<BR> ...,1451867210012_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-04 07:56:00,重大要聞,yahoo股市,工商時報晨訊,2016-01-04,◆ 千億 科技 預算 ， 張善政 喊 加碼 ， 主計 總處 初步 匡列 1 061 億 ...,up
289,時報資訊,0,2016年開紅盤日，台股狂洩逾200點！儘管2015年最後一個交易日拉尾盤，守住8300點，...,1451879808470_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-04 11:34:00,重大要聞,yahoo股市,《盤中解析》紅盤日變臉，台股狂洩逾200點,2016-01-04,2016 年開 紅盤 日 ， 台股 狂洩 逾 200 點 ！ 儘管 2015 年 最後 一個...,up


In [30]:
print(len(df_company[df_company['label5566']=='even']))
df_company[df_company['label5566']=='even'].head(2)

911


,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,post_time2,content2,label5566
3121,鉅亨網,0,受到月線反壓、外資籌碼面持續偏保守的影響下，台股期現貨今(20)日同步下壓，加權指數開低走低...,1453258680867_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-20 10:42:00,重大要聞,yahoo股市,台股盤中─個股漲勢稀落 指數一度大跌百點 台指期難拉高結算,2016-01-20,受到 月線 反壓 、 外資 籌碼面 持續 偏 保守 的 影響 下 ， 台股 期 現貨 今 (...,even
3192,Moneydj理財網,0,1.歐美股市走穩，不過受期指結算影響，台股週三開低，加上港股重挫，指數震盪走低，盤面上部份生...,1453281180692_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-20 16:16:00,重大要聞,yahoo股市,日盛投顧：台股短線參考區間7600-7900,2016-01-20,1 . 歐美 股市 走穩 ， 不過 受 期指 結算 影響 ， 台股 週三 開低 ， 加上 港...,even


In [31]:
df_company = df_company[df_company['label5566'].str.contains('up|down|even')]

In [32]:
import re
features = []   # features=list()
with open('finance.words.txt', 'r',encoding='utf-8') as data:
    for line in data:
#        line = re.sub('[a-zA-Z0-9\W]', '', line)                     # 把數字英文去掉
        line = re.sub('[0-9]', '', line)                              # 把數字去掉
        features.append(line.replace('\n', '').replace(' ', ''))      # 空格 \n去掉

print(len(features))
print(type(features))
features[:10]

3405
<class 'list'>


['營收', '交易', '成長', '上漲', '外資', '法人', '增加', '下跌', '獲利', '股']

In [33]:
#df_keyword1 = pd.read_csv("final_higher_tf_idf_part.csv",encoding="utf-8") #上漲形容詞
#df_keyword2 = pd.read_csv("final_lower_tf_idf_part.csv",encoding="utf-8")  #下跌形容詞
#df_keyword = pd.concat([df_keyword1,df_keyword2])
#del df_keyword1,df_keyword2
#df_keyword.head()

In [34]:
#features = df_keyword['key'].to_numpy()
#features

In [35]:
from sklearn.feature_extraction.text import TfidfVectorizer

#features = [ '上漲','下跌','看好','走高','走低','漲停','跌停']
#features = features[:1000]
cv = TfidfVectorizer()   #預設有空格就一個feature
#cv = TfidfVectorizer(features) # 設定自己要的詞 

r = pd.SparseDataFrame(cv.fit_transform(df_company['content2']), 
                       df_company.index,
                       cv.get_feature_names(), 
                       default_fill_value=0)

r.fillna(value=0.0, inplace=True)
r.head(2)

,00,000,000000001,000000002,000000003,000000004,000000005,000000006,000000007,000000008,...,龍澤科,龍燈,龍華,龍邦,龍頭,龍頭廠,龍馬,龐培,龐大,龔明鑫
222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.025144,0.03842,0.0,0.0,0.0,0.0
289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0


In [36]:
# from sklearn.feature_extraction.text import CountVectorizer

# #features = [ '上漲','下跌','看好','走高','走低','漲停','跌停']
# #features = features[:1000]
# #cv = CountVectorizer()   #預設有空格就一個feature
# cv = CountVectorizer(vocabulary = features) # 設定自己要的詞 

# r = pd.SparseDataFrame(cv.fit_transform(df_company['content2']), 
#                        df_company.index,
#                        cv.get_feature_names(), 
#                        default_fill_value=0)

# r.head(2)

In [37]:
df_company2 = pd.concat([df_company,r], axis=1)
df_company2.head(2)

,author,comment_count,content,id,p_type,page_url,post_time,s_area_name,s_name,title,...,龍澤科,龍燈,龍華,龍邦,龍頭,龍頭廠,龍馬,龐培,龐大,龔明鑫
222,財訊快報,0,◆千億科技預算，張善政喊加碼，主計總處初步匡列1 061億、年增3%，可能再調高。<BR> ...,1451867210012_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-04 07:56:00,重大要聞,yahoo股市,工商時報晨訊,...,0.0,0.0,0.0,0.0,0.025144,0.03842,0.0,0.0,0.0,0.0
289,時報資訊,0,2016年開紅盤日，台股狂洩逾200點！儘管2015年最後一個交易日拉尾盤，守住8300點，...,1451879808470_N01,news,https://tw.stock.yahoo.com/news_content/url/d/...,2016-01-04 11:34:00,重大要聞,yahoo股市,《盤中解析》紅盤日變臉，台股狂洩逾200點,...,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0


In [38]:
df_company2 = df_company2[df_company2['label5566'].str.contains('up|down|even')]   #只取漲跌
Y = df_company2['label5566']
X = df_company2.drop(columns=['author','comment_count','content','id','p_type','page_url','post_time','s_area_name','s_name','title','post_time2','content2','label5566'])
X.fillna(value=0.0, inplace=True)
X.head(2)

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/pandas/core/frame.py:4034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  downcast=downcast, **kwargs)


,00,000,000000001,000000002,000000003,000000004,000000005,000000006,000000007,000000008,...,龍澤科,龍燈,龍華,龍邦,龍頭,龍頭廠,龍馬,龐培,龐大,龔明鑫
222,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.025144,0.03842,0.0,0.0,0.0,0.0
289,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,0.0,0.0,0.0


In [39]:
Y.head(2)

222    up
289    up
Name: label5566, dtype: object

In [40]:
X = X.to_numpy()
Y = Y.to_numpy()

In [41]:
#將X:features array, Y:lable array 都切成 1:4

validation_size = 0.20
seed = 7
X_train, X_validation, Y_train, Y_validation = model_selection.train_test_split(X, Y, test_size=validation_size, random_state=seed)
print(len(X_train),len(X_validation))

2468 618


In [42]:
model_RandomForest = RandomForestClassifier()
model_RandomForest.fit(X_train, Y_train)
print(model_RandomForest.score(X_train, Y_train))

predictions = model_RandomForest.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9846029173419774
0.5453074433656958
[[200  27  32]
 [ 79  78  31]
 [ 87  25  59]]
              precision    recall  f1-score   support

        down       0.55      0.77      0.64       259
        even       0.60      0.41      0.49       188
          up       0.48      0.35      0.40       171

   micro avg       0.55      0.55      0.55       618
   macro avg       0.54      0.51      0.51       618
weighted avg       0.55      0.55      0.53       618



In [43]:
model_DecisionTree = DecisionTreeClassifier()
model_DecisionTree.fit(X_train, Y_train)
print(model_DecisionTree.score(X_train, Y_train))

predictions = model_DecisionTree.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.9991896272285251
0.49029126213592233
[[142  65  52]
 [ 67  79  42]
 [ 51  38  82]]
              precision    recall  f1-score   support

        down       0.55      0.55      0.55       259
        even       0.43      0.42      0.43       188
          up       0.47      0.48      0.47       171

   micro avg       0.49      0.49      0.49       618
   macro avg       0.48      0.48      0.48       618
weighted avg       0.49      0.49      0.49       618



In [44]:
model_GaussianNB = GaussianNB()
model_GaussianNB.fit(X_train, Y_train)
print(model_GaussianNB.score(X_train, Y_train))

predictions = model_GaussianNB.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))

0.9789303079416531
0.6181229773462783
[[186  36  37]
 [ 57 107  24]
 [ 56  26  89]]
              precision    recall  f1-score   support

        down       0.62      0.72      0.67       259
        even       0.63      0.57      0.60       188
          up       0.59      0.52      0.55       171

   micro avg       0.62      0.62      0.62       618
   macro avg       0.62      0.60      0.61       618
weighted avg       0.62      0.62      0.62       618



In [45]:
model_MultinomialNB = MultinomialNB()
model_MultinomialNB.fit(X_train, Y_train)
print(model_MultinomialNB.score(X_train, Y_train) )

predictions = model_MultinomialNB.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
print(model_MultinomialNB.coef_)

0.48905996758508913
0.43042071197411
[[258   1   0]
 [180   8   0]
 [171   0   0]]
              precision    recall  f1-score   support

        down       0.42      1.00      0.59       259
        even       0.89      0.04      0.08       188
          up       0.00      0.00      0.00       171

   micro avg       0.43      0.43      0.43       618
   macro avg       0.44      0.35      0.23       618
weighted avg       0.45      0.43      0.27       618

[[ -8.52661519  -8.77600872 -10.84805106 ... -10.78413523 -10.35454929
  -10.58112836]
 [ -8.43574421  -9.24493159 -10.79100874 ... -10.79100874 -10.43069174
  -10.79100874]
 [ -8.65184194  -9.00283496 -10.78763601 ... -10.78763601 -10.58039072
  -10.78763601]]


/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [46]:
# 用10-Fold CV並且列出平均的效率


# 呼叫單個model MLP
model_LogisticRegression = LogisticRegression()
name = 'MLP'
kfold = model_selection.KFold(n_splits=5, random_state=seed)   #分割 10% cross validation

cv_results = model_selection.cross_val_score(model_LogisticRegression, X_train, Y_train, cv=kfold, scoring='accuracy')
#model用MLP()                cross valitation 

print(cv_results)
print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/home/barry/minico

[0.59716599 0.53441296 0.57287449 0.59432049 0.57200811]
MLP: 0.574156 (0.022458)


In [47]:
model_LogisticRegression = LogisticRegression()
model_LogisticRegression.fit(X_train, Y_train)
print(model_LogisticRegression.score(X_train, Y_train))

predictions = model_LogisticRegression.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
print(model_LogisticRegression.coef_)

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.8658833063209076
0.6003236245954693
[[229  17  13]
 [ 87  76  25]
 [ 91  14  66]]
              precision    recall  f1-score   support

        down       0.56      0.88      0.69       259
        even       0.71      0.40      0.52       188
          up       0.63      0.39      0.48       171

   micro avg       0.60      0.60      0.60       618
   macro avg       0.64      0.56      0.56       618
weighted avg       0.63      0.60      0.58       618

[[-0.50068096 -0.16799923  0.         ...  0.03823952  0.15358254
   0.15867836]
 [ 0.51422379 -0.11692183  0.         ... -0.02831604 -0.01077253
  -0.0744015 ]
 [ 0.06289332  0.29765136  0.         ... -0.01043889 -0.13215527
  -0.08350618]]


In [48]:
# # 用10-Fold CV並且列出平均的效率


# # 呼叫單個model MLP
# model_svclinear = SVC(kernel='linear')
# name = 'SVC'
# kfold = model_selection.KFold(n_splits=5, random_state=seed)   #分割 10% cross validation

# cv_results = model_selection.cross_val_score(model_svclinear, X_train, Y_train, cv=kfold, scoring='accuracy')
# #model用MLP()                cross valitation 

# print(cv_results)
# print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))

In [49]:
# model_svclinear = SVC(kernel='linear')
# model_svclinear.fit(X_train, Y_train)
# print(model_svclinear.score(X_train, Y_train))
# predictions = model_svclinear.predict(X_validation)
# print(accuracy_score(Y_validation, predictions))
# print(confusion_matrix(Y_validation, predictions))
# print(classification_report(Y_validation, predictions))
# print(model_svclinear.coef_)

In [50]:
from sklearn.svm import LinearSVC
# 用10-Fold CV並且列出平均的效率


# 呼叫單個model MLP
model_LinearSVC = LinearSVC()
#model_LinearSVC = LinearSVC(loss='hinge')
name = 'SVC'
kfold = model_selection.KFold(n_splits=5, random_state=seed)   #分割 10% cross validation

cv_results = model_selection.cross_val_score(model_LinearSVC, X_train, Y_train, cv=kfold, scoring='accuracy')
#model用MLP()                cross valitation 

print(cv_results)
print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))


[0.63765182 0.61740891 0.63562753 0.63691684 0.64705882]
SVC: 0.634933 (0.009653)


In [51]:
from sklearn.svm import LinearSVC
model_LinearSVC = LinearSVC()
model_LinearSVC.fit(X_train, Y_train)
print(model_LinearSVC.score(X_train, Y_train))
predictions = model_LinearSVC.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
print(model_LinearSVC.coef_)

0.9878444084278768
0.6699029126213593
[[210  28  21]
 [ 50 107  31]
 [ 54  20  97]]
              precision    recall  f1-score   support

        down       0.67      0.81      0.73       259
        even       0.69      0.57      0.62       188
          up       0.65      0.57      0.61       171

   micro avg       0.67      0.67      0.67       618
   macro avg       0.67      0.65      0.65       618
weighted avg       0.67      0.67      0.66       618

[[-1.77201016e-01 -3.92797773e-01  0.00000000e+00 ...  9.85097619e-02
   3.69053051e-01  2.72798923e-01]
 [ 3.93831349e-02  3.00853005e-01  0.00000000e+00 ... -1.00245308e-01
  -7.45075232e-02 -1.14263928e-01]
 [ 1.55598472e-01  1.18610140e-01  0.00000000e+00 ...  1.51788304e-18
  -2.75125177e-01 -1.59401299e-01]]


In [52]:
from sklearn.linear_model import SGDClassifier

# 用10-Fold CV並且列出平均的效率


# 呼叫單個model MLP
model_SGDClassifier = SGDClassifier(loss='hinge')
#model_SGDClassifier = SGDClassifier(loss='log')
name = 'SVC'
kfold = model_selection.KFold(n_splits=5, random_state=seed)   #分割 10% cross validation

cv_results = model_selection.cross_val_score(model_SGDClassifier, X_train, Y_train, cv=kfold, scoring='accuracy')
#model用MLP()                cross valitation 

print(cv_results)
print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))


/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)
/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDCla

[0.60728745 0.60121457 0.61336032 0.64097363 0.59432049]
SVC: 0.611431 (0.016067)


In [53]:
from sklearn.linear_model import SGDClassifier
model_SGDClassifier = SGDClassifier(loss='hinge')
model_SGDClassifier.fit(X_train, Y_train)
print(model_SGDClassifier.score(X_train, Y_train))
predictions = model_SGDClassifier.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
print(model_SGDClassifier.coef_)

/home/barry/miniconda3/envs/stockenv/lib/python3.6/site-packages/sklearn/linear_model/stochastic_gradient.py:166: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If both are left unset, they default to max_iter=5 and tol=None. If tol is not None, max_iter defaults to max_iter=1000. From 0.21, default max_iter will be 1000, and default tol will be 1e-3.
  FutureWarning)


0.9789303079416531
0.6601941747572816
[[208  24  27]
 [ 53 106  29]
 [ 60  17  94]]
              precision    recall  f1-score   support

        down       0.65      0.80      0.72       259
        even       0.72      0.56      0.63       188
          up       0.63      0.55      0.59       171

   micro avg       0.66      0.66      0.66       618
   macro avg       0.67      0.64      0.65       618
weighted avg       0.66      0.66      0.66       618

[[-0.70887179 -0.61597457  0.         ...  0.19792389  0.66309897
   0.45871428]
 [ 0.30686189  0.96039513  0.         ... -0.19792389 -0.23122967
  -0.22935714]
 [ 1.17212607 -0.29415821  0.         ...  0.         -0.54682957
  -0.45871428]]


In [54]:
model_MLP = MLPClassifier(hidden_layer_sizes=(100,), max_iter=200)
model_MLP.fit(X_train, Y_train)
print(model_MLP.score(X_train, Y_train))

predictions = model_MLP.predict(X_validation)
print(accuracy_score(Y_validation, predictions))
print(confusion_matrix(Y_validation, predictions))
print(classification_report(Y_validation, predictions))
print(model_MLP.coefs_)

0.9987844408427877
0.6844660194174758
[[206  29  24]
 [ 45 118  25]
 [ 54  18  99]]
              precision    recall  f1-score   support

        down       0.68      0.80      0.73       259
        even       0.72      0.63      0.67       188
          up       0.67      0.58      0.62       171

   micro avg       0.68      0.68      0.68       618
   macro avg       0.69      0.67      0.67       618
weighted avg       0.69      0.68      0.68       618

[array([[-3.79260394e-02, -2.50890663e-02,  2.72390547e-02, ...,
         3.13629211e-02,  5.07918750e-02, -3.50392806e-02],
       [-1.72067462e-02, -2.16855338e-02,  2.01650400e-02, ...,
         5.31639776e-02,  5.41204151e-02, -2.86748735e-02],
       [ 2.25761887e-36, -1.02647245e-36,  3.72699233e-37, ...,
         2.16227467e-36, -2.74917598e-37,  1.27119691e-37],
       ...,
       [ 5.88556385e-02,  5.84365258e-02, -3.76544852e-02, ...,
        -6.82831095e-02, -5.93482004e-02,  6.80894515e-02],
       [ 2.81092360e-02,  

In [55]:
# 用10-Fold CV並且列出平均的效率


# 呼叫單個model MLP
model_MLP = MLPClassifier(hidden_layer_sizes=(100, ),max_iter=200)
name = 'MLP'
kfold = model_selection.KFold(n_splits=5, random_state=seed)   #分割 10% cross validation

cv_results = model_selection.cross_val_score(model_MLP, X_train, Y_train, cv=kfold, scoring='accuracy')
#model用MLP()                cross valitation 

print(cv_results)
print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))

[0.62753036 0.62753036 0.6417004  0.67545639 0.67139959]
MLP: 0.648723 (0.020864)


In [57]:
# 用10-Fold CV並且列出平均的效率


# 呼叫單個model MLP
model_MLP = MLPClassifier(hidden_layer_sizes=(100, ),max_iter=200)
name = 'MLP'
kfold = model_selection.KFold(n_splits=5, random_state=seed)   #分割 10% cross validation

cv_results = model_selection.cross_val_score(model_MLP, X, Y, cv=kfold, scoring='accuracy')
#model用MLP()                cross valitation 

print(cv_results)
print("%s: %f (%f)" % (name, cv_results.mean(), cv_results.std()))

[0.20064725 0.28525122 0.44408428 0.37277147 0.35656402]
MLP: 0.331864 (0.082786)


In [56]:
#df_company2.to_csv('5pa.csv')